# Imports

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None) 
import numpy as np
import re
from collections import defaultdict

# Gameplan

The following spreadsheets need to be cleaned and merged together:

    - players_yahooleague : contains all players from yahoo fantasy for every season, this is the base sheet that I'll be working off of
    - player_keys_raw : all the online ids for each fantasy api for each player
    - stats_team : team statistics to be used for defenses
    - stats_def: defensive statistics for defenses
    - conversions: defensive statistics for defenses
    - stats_player: player statistics to be used for granular player statistics
    - stats_yahooleague: Yahoo fantasy stats for every season, includes season end score
    - depth_14to24 : Depth charts from 14 to 24 season
        -depth_2025 : Depth chart for 2025 season -- columns and layour are a little different than other seasons
    -coaches: all head offensive and defensive coaches for every team for every year

**The first order of business will be finding the common IDs across all datasets in order to merge with unique keys, some are missing from some sets**



In [69]:
yahoo_players = pd.read_csv('Raw/players_yahooleague_raw_14to25.csv')
yahoo_players.head()
yahoo_players = yahoo_players.drop(columns='Unnamed: 0')
yahoo_players.head()


,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season
0,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2014
1,9,TE,nfl.p.3950,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,False,False,False,0,Tony,Tony,Tony Gonzalez,Gonzalez,3950,331.p.3950,O,TE,https://sports.yahoo.com/nfl/players/3950,2014
2,4,QB,nfl.p.4256,Den,Denver Broncos,nfl.t.7,https://sports.yahoo.com/nfl/teams/denver/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,False,False,False,0,Peyton,Peyton,Peyton Manning,Manning,4256,331.p.4256,O,QB,https://sports.yahoo.com/nfl/players/4256,2014
3,4,K,nfl.p.4269,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['K'],small,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,False,False,False,0,Phil,Phil,Phil Dawson,Dawson,4269,331.p.4269,K,K,https://sports.yahoo.com/nfl/players/4269,2014
4,10,QB,nfl.p.4416,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,False,False,False,0,Matt,Matt,Matt Hasselbeck,Hasselbeck,4416,331.p.4416,O,QB,https://sports.yahoo.com/nfl/players/4416,2014


In [70]:
master_ids = pd.read_csv('Raw/player_keys_raw.csv')
master_ids.head()
master_ids = master_ids.drop(columns='Unnamed: 0')
master_ids.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
0,17030,3c76cab3-3df2-43dd-acaa-57e055bd32d0,24755.0,00-0040676,133244.0,12522.0,58203.0,4688380.0,NaN,NaN,3168422.0,WardCa00,NaN,16997.0,NaN,1730.0,41786.0,0.0,25323.0,NaN,Cam Ward,cam ward,QB,TEN,2002-05-25,23.4,2025.0,1.0,1.0,1.0,NaN,74.0,219.0,Miami (FL),2025
1,17031,270e09bc-8bf8-44b0-87ed-1fd014de4ab7,25968.0,00-0040668,131396.0,12524.0,58349.0,4432762.0,NaN,NaN,26710004.0,SandSh00,NaN,18479.0,NaN,1731.0,41930.0,0.0,26079.0,NaN,Shedeur Sanders,shedeur sanders,QB,CLE,2002-02-07,23.7,2025.0,5.0,6.0,144.0,NaN,74.0,212.0,Colorado,2025
2,17032,53848cba-bcad-4cd0-bb35-0a1f6ed111d1,23160.0,00-0040691,146409.0,12508.0,58227.0,4689114.0,NaN,NaN,26694019.0,DartJa00,NaN,18574.0,NaN,1732.0,41810.0,0.0,26082.0,NaN,Jaxson Dart,jaxson dart,QB,NYG,2003-05-13,22.4,2025.0,1.0,25.0,25.0,NaN,74.0,223.0,Ole Miss,2025
3,17033,b0eb5b67-4ef2-462d-bfe7-e73bcce1f6b0,22884.0,00-0040673,146590.0,12510.0,58297.0,4432734.0,NaN,NaN,26694943.0,MilrJa00,NaN,18477.0,NaN,1733.0,41882.0,0.0,25882.0,NaN,Jalen Milroe,jalen milroe,QB,SEA,2002-12-13,22.8,2025.0,3.0,28.0,92.0,NaN,74.0,216.0,Alabama,2025
4,17034,d89f0384-21a6-44e4-8aa6-324cb2103bf6,23680.0,00-0040234,127253.0,12500.0,58436.0,4889929.0,NaN,NaN,26746721.0,EwerQu00,NaN,18505.0,NaN,1734.0,42037.0,0.0,26048.0,NaN,Quinn Ewers,quinn ewers,QB,MIA,2003-05-13,22.4,2025.0,7.0,15.0,231.0,NaN,74.0,209.0,Texas,2025


In [71]:
# Check how many of the master list are missing yahoo IDs

master_ids[master_ids['yahoo_id'].isna()]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
0,17030,3c76cab3-3df2-43dd-acaa-57e055bd32d0,24755.0,00-0040676,133244.0,12522.0,58203.0,4688380.0,NaN,NaN,3168422.0,WardCa00,NaN,16997.0,NaN,1730.0,41786.0,0.0,25323.0,NaN,Cam Ward,cam ward,QB,TEN,2002-05-25,23.4,2025.0,1.0,1.0,1.0,NaN,74.0,219.0,Miami (FL),2025
1,17031,270e09bc-8bf8-44b0-87ed-1fd014de4ab7,25968.0,00-0040668,131396.0,12524.0,58349.0,4432762.0,NaN,NaN,26710004.0,SandSh00,NaN,18479.0,NaN,1731.0,41930.0,0.0,26079.0,NaN,Shedeur Sanders,shedeur sanders,QB,CLE,2002-02-07,23.7,2025.0,5.0,6.0,144.0,NaN,74.0,212.0,Colorado,2025
2,17032,53848cba-bcad-4cd0-bb35-0a1f6ed111d1,23160.0,00-0040691,146409.0,12508.0,58227.0,4689114.0,NaN,NaN,26694019.0,DartJa00,NaN,18574.0,NaN,1732.0,41810.0,0.0,26082.0,NaN,Jaxson Dart,jaxson dart,QB,NYG,2003-05-13,22.4,2025.0,1.0,25.0,25.0,NaN,74.0,223.0,Ole Miss,2025
3,17033,b0eb5b67-4ef2-462d-bfe7-e73bcce1f6b0,22884.0,00-0040673,146590.0,12510.0,58297.0,4432734.0,NaN,NaN,26694943.0,MilrJa00,NaN,18477.0,NaN,1733.0,41882.0,0.0,25882.0,NaN,Jalen Milroe,jalen milroe,QB,SEA,2002-12-13,22.8,2025.0,3.0,28.0,92.0,NaN,74.0,216.0,Alabama,2025
4,17034,d89f0384-21a6-44e4-8aa6-324cb2103bf6,23680.0,00-0040234,127253.0,12500.0,58436.0,4889929.0,NaN,NaN,26746721.0,EwerQu00,NaN,18505.0,NaN,1734.0,42037.0,0.0,26048.0,NaN,Quinn Ewers,quinn ewers,QB,MIA,2003-05-13,22.4,2025.0,7.0,15.0,231.0,NaN,74.0,209.0,Texas,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12140,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025
12141,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025
12142,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025
12143,990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bubby Brister,bubby brister,QB,KCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025


Missing ids (approx 6700) look to be rookies this year or players who played a very long time ago. Also noticed there are a handful of fantasy positions on list that arent in league like defensive individual players

In [72]:
# See how many positions are fantasy relevent after dropping defenses, make a raw copy first for later data validation

master_raw = master_ids.copy()

master_ids = master_ids[master_ids['position'].isin(['QB', 'RB', 'WR', 'TE', 'PK'])]
master_ids['position'].value_counts()

position
WR    2038
RB    1587
TE     948
QB     666
PK     276
Name: count, dtype: int64

In [119]:
master_raw[~master_raw['yahoo_id'].isin(master_raw['stats_id'])]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
402,16634,ffd59710-0582-11ef-823c-c3942ac17525,23957.0,00-0039521,101130.0,11921.0,57639.0,4428811.0,41202.0,NaN,3131562.0,WeavXa00,NaN,18041.0,NaN,NaN,NaN,0.0,25032.0,1180538.0,Xavier Weaver,xavier weaver,WR,ARI,2000-10-12,25.0,2024.0,NaN,NaN,NaN,NaN,73.0,185.0,Colorado,2025
741,16195,ea3d1393-8c3b-4e81-a519-f636d28d692e,24354.0,00-0038393,99711.0,9486.0,56024.0,4428850.0,40201.0,NaN,3122375.0,WickDo00,dontayvion-wicks-1,16658.0,NaN,1448.0,NaN,0.0,23253.0,1172428.0,Dontayvion Wicks,dontayvion wicks,WR,GBP,2001-06-16,24.3,2023.0,5.0,24.0,159.0,NaN,73.0,206.0,Virginia,2025
766,16220,5d4a5e4f-7c74-4b4c-b8e5-33bb831e3057,23821.0,00-0038394,77888.0,10220.0,56027.0,4362523.0,40180.0,NaN,2962073.0,MallWi00,will-mallory-1,17014.0,NaN,NaN,NaN,0.0,23143.0,1121873.0,Will Mallory,will mallory,TE,IND,1999-06-22,26.3,2023.0,5.0,27.0,162.0,NaN,76.0,239.0,Miami (FL),2025
1030,16535,64f9db10-e766-11ed-8199-d58dac5a4f8e,25588.0,00-0038850,150239.0,11096.0,56570.0,4568018.0,40327.0,NaN,3122209.0,McLeTK00,NaN,17575.0,NaN,NaN,17575.0,0.0,NaN,1161726.0,TK McLendon,tk mclendon,DE,FA,1999-02-27,26.6,2023.0,NaN,NaN,NaN,NaN,76.0,275.0,Eastern Kentucky,2025
1142,15759,2995df34-ee81-4320-be1e-1c497b83c927,22895.0,00-0037614,97400.0,8147.0,54509.0,4567096.0,33999.0,NaN,3121171.0,MetcJo00,john-metchie-1,16069.0,NaN,NaN,34063.0,0.0,23160.0,1160348.0,John Metchie,john metchie,WR,PHI,2000-07-18,25.2,2022.0,2.0,12.0,44.0,NaN,72.0,195.0,Alabama,2025
1812,15677,7656034d-8647-49e5-b0d4-b708423042ee,23339.0,00-0036696,NaN,7903.0,NaN,NaN,33837.0,NaN,2247169.0,NaN,NaN,15561.0,NaN,NaN,NaN,0.0,NaN,921082.0,Briley Moore-McKinney,briley moore-mckinney,TE,FA,1998-01-13,24.1,2021.0,NaN,NaN,NaN,NaN,76.0,250.0,Kansas State,2025
2499,14325,c66e5f22-10ad-41fb-9fde-52598df5fba7,NaN,00-0034931,49816.0,6046.0,48054.0,3133361.0,32190.0,NaN,3116707.0,DineJo01,NaN,13769.0,14307.0,NaN,NaN,0.0,21068.0,NaN,Joe Dineen,joe dineen,LB,FA,1996-03-26,29.6,2019.0,NaN,NaN,NaN,NaN,74.0,235.0,Kansas,2025
2606,14649,55c1614f-c234-45d9-baf5-1cc808faaa4d,NaN,00-0035340,48493.0,6538.0,48469.0,3115334.0,32436.0,NaN,2133524.0,HendDe04,NaN,14076.0,NaN,NaN,140760.0,829992.0,21269.0,NaN,Dewayne Hendrix,dewayne hendrix,DE,FA,1995-06-03,25.9,2019.0,NaN,NaN,NaN,NaN,76.0,270.0,Pittsburgh,2025
2676,15678,457123e2-aeb5-41c7-b2a8-ddc42c04f977,NaN,00-0035518,48665.0,6602.0,48668.0,3115492.0,32424.0,NaN,NaN,HoyeBr01,NaN,14203.0,NaN,NaN,52366.0,0.0,21358.0,835503.0,Braxton Hoyett,braxton hoyett,DT,NOS,1996-02-05,26.1,2019.0,NaN,NaN,NaN,NaN,74.0,302.0,Mississippi State,2025
2737,13643,1dea7b00-f200-470f-8912-ca9d87ceb7e2,NaN,00-0034068,28580.0,5164.0,46326.0,3128251.0,31397.0,NaN,2924881.0,CobbSi00,NaN,12480.0,13386.0,NaN,NaN,0.0,20107.0,NaN,Simmie Cobbs,simmie cobbs,WR,FA,1995-08-25,30.1,2018.0,NaN,NaN,NaN,NaN,75.0,220.0,Indiana,2025


In [124]:
yahoo_players[(~yahoo_players['player_id'].isin(master_raw['stats_id'])) & (~yahoo_players['player_id'].isin(master_raw['yahoo_id'])) & (yahoo_players['primary_position'] != 'DEF')]

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season
423,4,TE,nfl.p.25050,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/SKdKxY1l7uWA...,https://s.yimg.com/iu/api/res/1.2/SKdKxY1l7uWA...,False,False,False,0,Martell,Martell,Martell Webb,Webb,25050,331.p.25050,O,TE,https://sports.yahoo.com/nfl/players/25050,2014
432,5,"WR,TE",nfl.p.25123,Mia,Miami Dolphins,nfl.t.15,https://sports.yahoo.com/nfl/teams/miami/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Raymond,Raymond,Raymond Webber,Webber,25123,331.p.25123,O,WR,https://sports.yahoo.com/nfl/players/25123,2014
450,10,RB,nfl.p.25303,NE,New England Patriots,nfl.t.17,https://sports.yahoo.com/nfl/teams/new-england/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Eric,Eric,Eric Kettani,Kettani,25303,331.p.25303,O,RB,https://sports.yahoo.com/nfl/players/25303,2014
456,9,TE,nfl.p.25370,Chi,Chicago Bears,nfl.t.3,https://sports.yahoo.com/nfl/teams/chicago/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/eYc8ZhQunK1....,https://s.yimg.com/iu/api/res/1.2/eYc8ZhQunK1....,False,False,False,0,Patrick,Patrick,Patrick Scales,Scales,25370,331.p.25370,O,TE,https://sports.yahoo.com/nfl/players/25370,2014
561,9,WR,nfl.p.25973,Buf,Buffalo Bills,nfl.t.2,https://sports.yahoo.com/nfl/teams/buffalo/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/8bC.eqBYxrrj...,https://s.yimg.com/iu/api/res/1.2/8bC.eqBYxrrj...,False,False,False,0,Chris,Chris,Chris Summers,Summers,25973,331.p.25973,O,WR,https://sports.yahoo.com/nfl/players/25973,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14058,8,TE,nfl.p.42592,Det,Detroit Lions,nfl.t.8,https://sports.yahoo.com/nfl/teams/detroit/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Gunnar,Gunnar,Gunnar Oakes,Oakes,42592,461.p.42592,O,TE,https://sports.yahoo.com/nfl/players/42592,2025
14059,14,RB,nfl.p.42596,NE,New England Patriots,nfl.t.17,https://sports.yahoo.com/nfl/teams/new-england/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Shane,Shane,Shane Watts,Watts,42596,461.p.42596,O,RB,https://sports.yahoo.com/nfl/players/42596,2025
14060,11,WR,nfl.p.42597,NO,New Orleans Saints,nfl.t.18,https://sports.yahoo.com/nfl/teams/new-orleans/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Roderick,Roderick,Roderick Daniels Jr.,Daniels Jr.,42597,461.p.42597,O,WR,https://sports.yahoo.com/nfl/players/42597,2025
14062,14,WR,nfl.p.42602,NE,New England Patriots,nfl.t.17,https://sports.yahoo.com/nfl/teams/new-england/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Phil,Phil,Phil Lutz,Lutz,42602,461.p.42602,O,WR,https://sports.yahoo.com/nfl/players/42602,2025


In [120]:
# How many are left?
master_ids['stats_id'].isna().sum()

# 4000 missing IDs were removed this way, good start

np.int64(49)

In [74]:
# Quick visual check of missing ids left
master_ids[master_ids['yahoo_id'].isna()].head(10)

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
0,17030,3c76cab3-3df2-43dd-acaa-57e055bd32d0,24755.0,00-0040676,133244.0,12522.0,58203.0,4688380.0,NaN,NaN,3168422.0,WardCa00,NaN,16997.0,NaN,1730.0,41786.0,0.0,25323.0,NaN,Cam Ward,cam ward,QB,TEN,2002-05-25,23.4,2025.0,1.0,1.0,1.0,NaN,74.0,219.0,Miami (FL),2025
1,17031,270e09bc-8bf8-44b0-87ed-1fd014de4ab7,25968.0,00-0040668,131396.0,12524.0,58349.0,4432762.0,NaN,NaN,26710004.0,SandSh00,NaN,18479.0,NaN,1731.0,41930.0,0.0,26079.0,NaN,Shedeur Sanders,shedeur sanders,QB,CLE,2002-02-07,23.7,2025.0,5.0,6.0,144.0,NaN,74.0,212.0,Colorado,2025
2,17032,53848cba-bcad-4cd0-bb35-0a1f6ed111d1,23160.0,00-0040691,146409.0,12508.0,58227.0,4689114.0,NaN,NaN,26694019.0,DartJa00,NaN,18574.0,NaN,1732.0,41810.0,0.0,26082.0,NaN,Jaxson Dart,jaxson dart,QB,NYG,2003-05-13,22.4,2025.0,1.0,25.0,25.0,NaN,74.0,223.0,Ole Miss,2025
3,17033,b0eb5b67-4ef2-462d-bfe7-e73bcce1f6b0,22884.0,00-0040673,146590.0,12510.0,58297.0,4432734.0,NaN,NaN,26694943.0,MilrJa00,NaN,18477.0,NaN,1733.0,41882.0,0.0,25882.0,NaN,Jalen Milroe,jalen milroe,QB,SEA,2002-12-13,22.8,2025.0,3.0,28.0,92.0,NaN,74.0,216.0,Alabama,2025
4,17034,d89f0384-21a6-44e4-8aa6-324cb2103bf6,23680.0,00-0040234,127253.0,12500.0,58436.0,4889929.0,NaN,NaN,26746721.0,EwerQu00,NaN,18505.0,NaN,1734.0,42037.0,0.0,26048.0,NaN,Quinn Ewers,quinn ewers,QB,MIA,2003-05-13,22.4,2025.0,7.0,15.0,231.0,NaN,74.0,209.0,Texas,2025
5,17035,38b62d7f-4403-4e2b-8083-4d0cd08e81aa,25373.0,00-0040206,143862.0,12470.0,58394.0,4683423.0,NaN,NaN,26699092.0,LeonRi02,NaN,18513.0,NaN,1735.0,41968.0,0.0,24992.0,NaN,Riley Leonard,riley leonard,QB,IND,2002-09-13,23.1,2025.0,6.0,13.0,189.0,NaN,76.0,213.0,Notre Dame,2025
6,17036,b8a1b579-3a7e-4858-960b-fbfaca8de613,25972.0,00-0040203,122599.0,12511.0,58390.0,4429955.0,NaN,NaN,3162310.0,HowaWi01,NaN,18512.0,NaN,1736.0,41989.0,0.0,25884.0,NaN,Will Howard,will howard,QB,PIT,2001-09-24,24.1,2025.0,6.0,9.0,185.0,NaN,76.0,236.0,Ohio State,2025
7,17037,299d4fe8-188c-43d1-95d3-8a8f103da5c5,23150.0,00-0040704,86272.0,12486.0,58299.0,4427238.0,NaN,NaN,3121534.0,GabrDi00,NaN,18496.0,NaN,1737.0,41861.0,0.0,23177.0,NaN,Dillon Gabriel,dillon gabriel,QB,CLE,2000-12-28,24.8,2025.0,3.0,30.0,94.0,NaN,71.0,205.0,Oregon,2025
8,17038,451c7689-4fc2-410a-a33f-22dfa2ab37d4,23066.0,00-0040014,145053.0,12494.0,58386.0,4433971.0,NaN,NaN,26702687.0,McCoKy00,NaN,18498.0,NaN,1738.0,41971.0,0.0,26085.0,NaN,Kyle McCord,kyle mccord,QB,PHI,2002-09-19,23.1,2025.0,6.0,5.0,181.0,NaN,75.0,218.0,Syracuse,2025
9,17039,94b8f9e1-3109-4429-8f71-4f79d7588237,23096.0,00-0040743,83537.0,12545.0,58245.0,4360689.0,NaN,NaN,2964064.0,ShouTy00,NaN,17037.0,NaN,1739.0,41825.0,0.0,23187.0,NaN,Tyler Shough,tyler shough,QB,NOS,1999-09-28,26.1,2025.0,2.0,8.0,40.0,NaN,77.0,219.0,Louisville,2025


In [75]:
# Rookies missing are on the yahoo list, will make a yahoo ID map and use to add to master list

yahoo_id_maps = yahoo_players.set_index('name.full')['player_id'].to_dict()
master_ids.loc[master_ids['yahoo_id'].isna(), 'yahoo_id'] = master_ids.loc[master_ids['yahoo_id'].isna(), 'name'].map(yahoo_id_maps)

In [76]:
# Quick visual check of missing ids left
master_ids[master_ids['yahoo_id'].isna()].head(10)

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
29,17059,42753bb4-a416-4d40-bf81-f20d2181be2f,25984.0,00-0040198,157792.0,12495.0,58384.0,4711533.0,NaN,NaN,28873887.0,GordOl00,NaN,18450.0,NaN,1759.0,41969.0,0.0,25953.0,NaN,Ollie Gordon,ollie gordon,RB,MIA,2004-01-15,21.8,2025.0,6.0,3.0,179.0,NaN,74.0,225.0,Oklahoma State,2025
31,17061,14aedd06-41a6-41db-8c5e-05483f91570d,27285.0,00-0040238,158290.0,12544.0,58441.0,4911851.0,NaN,NaN,28877884.0,AlleLe01,NaN,18476.0,NaN,1761.0,42007.0,0.0,25986.0,NaN,LeQuint Allen,lequint allen,RB,JAC,2004-08-05,21.2,2025.0,7.0,20.0,236.0,NaN,72.0,201.0,Syracuse,2025
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025
40,17070,a4c21977-c9d1-40f8-abee-0862dfab9195,27016.0,00-0040735,157170.0,12519.0,58244.0,4685278.0,NaN,NaN,28874444.0,BurdLu00,NaN,18548.0,NaN,1770.0,41824.0,0.0,26044.0,NaN,Luther Burden,luther burden,WR,CHI,2003-12-12,21.8,2025.0,2.0,7.0,39.0,NaN,72.0,210.0,Missouri,2025
42,17072,a45328a1-7d0e-451a-8176-ea30f0e1c86f,26215.0,00-0040727,122676.0,12509.0,58260.0,4686612.0,NaN,NaN,3162525.0,HarrTr03,NaN,18478.0,NaN,1772.0,41840.0,0.0,25925.0,NaN,Tre Harris,tre harris,WR,LAC,2002-02-28,23.6,2025.0,2.0,23.0,55.0,NaN,75.0,210.0,Ole Miss,2025
54,17084,f2e485f8-ba20-4c1c-b279-41dcdcb28d03,27186.0,00-0040653,129639.0,12475.0,58443.0,4431479.0,NaN,NaN,3169795.0,WhitRi01,NaN,18560.0,NaN,NaN,42042.0,0.0,25010.0,NaN,Ricky White,ricky white,WR,FA,2002-02-06,23.7,2025.0,7.0,22.0,238.0,NaN,73.0,184.0,UNLV,2025
57,17087,483606b2-c0c9-4da2-8fcf-73d2615756c7,27288.0,00-0040581,146547.0,12541.0,58313.0,4432775.0,NaN,NaN,26694424.0,ThorDo00,NaN,18558.0,NaN,1787.0,41892.0,0.0,26102.0,NaN,Dont'e Thornton,donte thornton,WR,LVR,2002-11-30,22.9,2025.0,4.0,6.0,108.0,NaN,77.0,205.0,Tennessee,2025
58,17088,277f6ff0-2442-11f0-ac20-e1591f73b631,27222.0,00-0040311,98005.0,12860.0,58747.0,4426535.0,NaN,NaN,3124276.0,WeasTh00,NaN,18645.0,NaN,NaN,42161.0,0.0,23169.0,NaN,Theo Wease,theo wease,WR,MIA,2001-06-25,24.3,2025.0,NaN,NaN,NaN,NaN,74.0,210.0,Missouri,2025
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025


Quick visual inspection shows that at least some players are not matchijng beacus eof name suffixes like JR and Sr.

In [77]:
# Check first name on list with known suffix and see if it shows up -- ollie gordon is actuall Ollie GOrdon II
yahoo_players[yahoo_players['name.full'] == 'Ollie Gordon II']

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season
13835,12,RB,nfl.p.41969,Mia,Miami Dolphins,nfl.t.15,https://sports.yahoo.com/nfl/teams/miami/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/eDlltI2Ax54R...,https://s.yimg.com/iu/api/res/1.2/eDlltI2Ax54R...,False,False,False,0,Ollie,Ollie,Ollie Gordon II,Gordon II,41969,461.p.41969,O,RB,https://sports.yahoo.com/nfl/players/41969,2025


Need to check if standardizing names (removing suffixes) would create a variety of dupes. IE two John Smith (suffix) would be just john smith

In [78]:
# Create name cleaning funciton
def clean_name(name):
    if pd.isna(name) or name is None:
        return name
    return re.sub(r'\s+(Jr\.?|Sr\.?|III|II|IV|V)(\s|$|,)', '', str(name), flags=re.IGNORECASE).strip()

In [79]:
# create function to cehck if dupes will be created and list where
def check_cleaning_duplicates(original_dict, clean_function):
    """Check if cleaning will create duplicate keys"""
    
    # Map: cleaned_name -> list of original names
    cleaned_to_original = defaultdict(list)
    
    for original_name in original_dict.keys():
        cleaned = clean_function(original_name)
        cleaned_to_original[cleaned].append(original_name)
    
    # Find cleaned names that have multiple originals
    duplicates = {k: v for k, v in cleaned_to_original.items() if len(v) > 1}
    
    if duplicates:
        print(f"Found {len(duplicates)} names that will become duplicates after cleaning:\n")
        for cleaned_name, original_names in duplicates.items():
            print(f'{cleaned_name}')
            for orig in original_names:
                print(f"    - '{orig}' (ID: {original_dict[orig]})")
            print()
        return duplicates
    else:
        print("No duplicates will be created after cleaning")
        return {}

In [80]:
# Run function

check_cleaning_duplicates(yahoo_id_maps, clean_name)

Found 1 names that will become duplicates after cleaning:

Frank Gore
    - 'Frank Gore' (ID: 7241)
    - 'Frank Gore Jr.' (ID: 41243)



{'Frank Gore': ['Frank Gore', 'Frank Gore Jr.']}

Gore and gore Jr are not missing any yahoo IDs so the code should not edit these IDs in any way, everything esle is covered

In [81]:
# Create mapping with cleaned names
cln_yahoo_id_maps = {clean_name(key): value for key, value in yahoo_id_maps.items()}

# Add ID maps woth standardized names
master_ids.loc[master_ids['yahoo_id'].isna(), 'yahoo_id'] = master_ids.loc[master_ids['yahoo_id'].isna(), 'name'].map(cln_yahoo_id_maps)

In [82]:
# Any missing yahoo ID values are now because those players were not in the yahoo ID Set, can drop these rows
master_ids = master_ids.dropna(subset=['yahoo_id'])
master_ids['yahoo_id'] = master_ids['yahoo_id'].astype(int)
master_ids.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3147 entries, 0 to 12138
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mfl_id            3147 non-null   int64  
 1   sportradar_id     3051 non-null   object 
 2   fantasypros_id    2485 non-null   float64
 3   gsis_id           2891 non-null   object 
 4   pff_id            2801 non-null   float64
 5   sleeper_id        2822 non-null   float64
 6   nfl_id            2773 non-null   float64
 7   espn_id           2944 non-null   float64
 8   yahoo_id          3147 non-null   int64  
 9   fleaflicker_id    297 non-null    float64
 10  cbs_id            3040 non-null   float64
 11  pfr_id            2785 non-null   object 
 12  cfbref_id         1362 non-null   object 
 13  rotowire_id       3108 non-null   float64
 14  rotoworld_id      2025 non-null   float64
 15  ktc_id            393 non-null    float64
 16  stats_id          3098 non-null   float64
 17 

In [83]:
# check that player like ollie gordon remain

display(master_ids[master_ids['name'] == 'Ollie Gordon'])
display(master_ids[master_ids['name'] == 'Luther Burden'])



,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
29,17059,42753bb4-a416-4d40-bf81-f20d2181be2f,25984.0,00-0040198,157792.0,12495.0,58384.0,4711533.0,41969,NaN,28873887.0,GordOl00,NaN,18450.0,NaN,1759.0,41969.0,0.0,25953.0,NaN,Ollie Gordon,ollie gordon,RB,MIA,2004-01-15,21.8,2025.0,6.0,3.0,179.0,NaN,74.0,225.0,Oklahoma State,2025


,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
40,17070,a4c21977-c9d1-40f8-abee-0862dfab9195,27016.0,00-0040735,157170.0,12519.0,58244.0,4685278.0,41824,NaN,28874444.0,BurdLu00,NaN,18548.0,NaN,1770.0,41824.0,0.0,26044.0,NaN,Luther Burden,luther burden,WR,CHI,2003-12-12,21.8,2025.0,2.0,7.0,39.0,NaN,72.0,210.0,Missouri,2025


In [84]:
# Check how many unqiue ids are in each dataframe

print(f' unqiue yahoo IDs - {yahoo_players["player_id"].nunique()}')
print(f' unqiue master IDs - {master_ids["yahoo_id"].nunique()}')

 unqiue yahoo IDs - 4263
 unqiue master IDs - 3100


In [85]:
# Look at which players are in the yahoo list but not in master ID

yahoo_players[~yahoo_players['player_id'].isin(master_ids['yahoo_id'])]


,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season
69,9,K,nfl.p.7434,Ten,Tennessee Titans,nfl.t.10,https://sports.yahoo.com/nfl/teams/tennessee/,['K'],small,https://s.yimg.com/iu/api/res/1.2/etIu4LtcHSr7...,https://s.yimg.com/iu/api/res/1.2/etIu4LtcHSr7...,False,False,False,0,Rob,Rob,Rob Bironas,Bironas,7434,331.p.7434,K,K,https://sports.yahoo.com/nfl/players/7434,2014
182,8,RB,nfl.p.9004,NYG,New York Giants,nfl.t.19,https://sports.yahoo.com/nfl/teams/ny-giants/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/xlu80ZICCzpp...,https://s.yimg.com/iu/api/res/1.2/xlu80ZICCzpp...,False,False,False,0,Peyton,Peyton,Peyton Hillis,Hillis,9004,331.p.9004,O,RB,https://sports.yahoo.com/nfl/players/9004,2014
188,4,RB,nfl.p.9039,Cin,Cincinnati Bengals,nfl.t.4,https://sports.yahoo.com/nfl/teams/cincinnati/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/e76NOWDVv5.v...,https://s.yimg.com/iu/api/res/1.2/e76NOWDVv5.v...,False,False,False,0,BenJarvus,BenJarvus,BenJarvus Green-Ellis,Green-Ellis,9039,331.p.9039,O,RB,https://sports.yahoo.com/nfl/players/9039,2014
348,12,WR,nfl.p.24746,Car,Carolina Panthers,nfl.t.29,https://sports.yahoo.com/nfl/teams/carolina/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/M8gJ9e0k7poq...,https://s.yimg.com/iu/api/res/1.2/M8gJ9e0k7poq...,False,False,False,0,Teddy,Teddy,Teddy Williams,Williams,24746,331.p.24746,O,WR,https://sports.yahoo.com/nfl/players/24746,2014
421,4,WR,nfl.p.25036,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,DeMarco,DeMarco,DeMarco Sampson,Sampson,25036,331.p.25036,O,WR,https://sports.yahoo.com/nfl/players/25036,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14091,12,DEF,nfl.p.100028,Was,Washington Commanders,nfl.t.28,https://sports.yahoo.com/nfl/teams/washington/,['DEF'],small,https://s.yimg.com/cv/apiv2/default/nfl/202009...,https://s.yimg.com/cv/apiv2/default/nfl/202009...,False,False,False,0,Washington,Washington,Washington Commanders,Commanders,100028,461.p.100028,DT,DEF,https://sports.yahoo.com/nfl/teams/washington/,2025
14092,14,DEF,nfl.p.100029,Car,Carolina Panthers,nfl.t.29,https://sports.yahoo.com/nfl/teams/carolina/,['DEF'],small,https://s.yimg.com/cv/ip/ap/default/120210/50x...,https://s.yimg.com/cv/ip/ap/default/120210/50x...,False,False,False,0,Carolina,Carolina,Carolina Panthers,Panthers,100029,461.p.100029,DT,DEF,https://sports.yahoo.com/nfl/teams/carolina/,2025
14093,8,DEF,nfl.p.100030,Jax,Jacksonville Jaguars,nfl.t.30,https://sports.yahoo.com/nfl/teams/jacksonville/,['DEF'],small,https://s.yimg.com/xe/ipt/50x50w.3.gif,https://s.yimg.com/xe/ipt/50x50w.3.gif,False,False,False,0,Jacksonville,Jacksonville,Jacksonville Jaguars,Jaguars,100030,461.p.100030,DT,DEF,https://sports.yahoo.com/nfl/teams/jacksonville/,2025
14094,7,DEF,nfl.p.100033,Bal,Baltimore Ravens,nfl.t.33,https://sports.yahoo.com/nfl/teams/baltimore/,['DEF'],small,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,False,False,False,0,Baltimore,Baltimore,Baltimore Ravens,Ravens,100033,461.p.100033,DT,DEF,https://sports.yahoo.com/nfl/teams/baltimore/,2025


These are known players, should have a yahoo ID. It does make sense that the DST players are not in the master list. Upon visual inspection, the first two players yahoo id is listed in the stat ID column. Will explore a little further before proceding

In [86]:
# Check season value counts
display(yahoo_players[~yahoo_players['player_id'].isin(master_ids['yahoo_id'])]['season'].value_counts())

# Visual inspection only shows 2014, check another season visually
yahoo_players[(~yahoo_players['player_id'].isin(master_ids['yahoo_id'])) & (yahoo_players['season'] == 2019) & (yahoo_players['primary_position'] != 'DEF')]


season
2014    195
2025    195
2016    193
2015    193
2017    191
2018    187
2022    162
2019    161
2023    160
2024    159
2020    144
2021    130
Name: count, dtype: int64

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season
6020,10,RB,nfl.p.25295,Was,Washington Commanders,nfl.t.28,https://sports.yahoo.com/nfl/teams/washington/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/yeTMhpBqUaCU...,https://s.yimg.com/iu/api/res/1.2/yeTMhpBqUaCU...,False,False,False,0,Nick,Nick,Nick Bellore,Bellore,25295,390.p.25295,O,RB,https://sports.yahoo.com/nfl/players/25295,2019
6021,6,TE,nfl.p.25370,Chi,Chicago Bears,nfl.t.3,https://sports.yahoo.com/nfl/teams/chicago/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/eYc8ZhQunK1....,https://s.yimg.com/iu/api/res/1.2/eYc8ZhQunK1....,False,False,False,0,Patrick,Patrick,Patrick Scales,Scales,25370,390.p.25370,O,TE,https://sports.yahoo.com/nfl/players/25370,2019
6056,6,TE,nfl.p.26050,Chi,Chicago Bears,nfl.t.3,https://sports.yahoo.com/nfl/teams/chicago/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/uILMLlKjh36G...,https://s.yimg.com/iu/api/res/1.2/uILMLlKjh36G...,False,False,False,0,Bradley,Bradley,Bradley Sowell,Sowell,26050,390.p.26050,O,TE,https://sports.yahoo.com/nfl/players/26050,2019
6072,12,TE,nfl.p.26583,Min,Minnesota Vikings,nfl.t.16,https://sports.yahoo.com/nfl/teams/minnesota/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/f0hoO_Wz9sSq...,https://s.yimg.com/iu/api/res/1.2/f0hoO_Wz9sSq...,False,False,False,0,Andrew,Andrew,Andrew DePaola,DePaola,26583,390.p.26583,O,TE,https://sports.yahoo.com/nfl/players/26583,2019
6125,12,TE,nfl.p.27214,Min,Minnesota Vikings,nfl.t.16,https://sports.yahoo.com/nfl/teams/minnesota/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/cED_NW3qVaNE...,https://s.yimg.com/iu/api/res/1.2/cED_NW3qVaNE...,False,False,False,0,Kevin,Kevin,Kevin McDermott,McDermott,27214,390.p.27214,O,TE,https://sports.yahoo.com/nfl/players/27214,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7074,6,WR,nfl.p.32639,Chi,Chicago Bears,nfl.t.3,https://sports.yahoo.com/nfl/teams/chicago/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Joe,Joe,Joe Walker,Walker,32639,390.p.32639,O,WR,https://sports.yahoo.com/nfl/players/32639,2019
7075,11,TE,nfl.p.32640,Sea,Seattle Seahawks,nfl.t.26,https://sports.yahoo.com/nfl/teams/seattle/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Jackson,Jackson,Jackson Harris,Harris,32640,390.p.32640,O,TE,https://sports.yahoo.com/nfl/players/32640,2019
7077,7,TE,nfl.p.32647,Pit,Pittsburgh Steelers,nfl.t.23,https://sports.yahoo.com/nfl/teams/pittsburgh/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Micky,Micky,Micky Crum,Crum,32647,390.p.32647,O,TE,https://sports.yahoo.com/nfl/players/32647,2019
7078,7,K,nfl.p.32650,Pit,Pittsburgh Steelers,nfl.t.23,https://sports.yahoo.com/nfl/teams/pittsburgh/,['K'],small,https://s.yimg.com/iu/api/res/1.2/qcLC3izfYduN...,https://s.yimg.com/iu/api/res/1.2/qcLC3izfYduN...,False,False,False,0,Cameron,Cameron,Cameron Nizialek,Nizialek,32650,390.p.32650,K,K,https://sports.yahoo.com/nfl/players/32650,2019


Upon further inspection, need to loop back to teh master raw lost6 and make sure my yahoo import didnt mess up some of its lines. A hadnful of the missing players lok to be long snappers or defenders but have offesnive posotions listed

In [111]:
# First change PK In master to K to match yahoo convention
master_raw['position'] = master_raw['position'].replace('PK', 'K')

# Stats ID looks to have more overlap, check the similarities here
print(f'stat ID col overlap - {len(yahoo_players[yahoo_players["player_id"].isin(master_raw["stats_id"])])}')
# Stats ID looks to have more overlap, check the similarities here
print(f'yahoo id ID col overlap - {len(yahoo_players[yahoo_players["player_id"].isin(master_raw["yahoo_id"])])}')

stat ID col overlap - 12050
yahoo id ID col overlap - 11544


In [114]:
master_raw['stats_id'].nunique()

9857

In [116]:
master_raw['yahoo_id'].nunique()

5482

In [ ]:
# Need to check where there might be postion mismatches in the positon column

positon_check = yahoo_players.merge(master_raw)

# yahoo_players[(yahoo_players["player_id"].isin(master_raw["stats_id"]))]

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season
0,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2014
1,9,TE,nfl.p.3950,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,False,False,False,0,Tony,Tony,Tony Gonzalez,Gonzalez,3950,331.p.3950,O,TE,https://sports.yahoo.com/nfl/players/3950,2014
2,4,QB,nfl.p.4256,Den,Denver Broncos,nfl.t.7,https://sports.yahoo.com/nfl/teams/denver/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,False,False,False,0,Peyton,Peyton,Peyton Manning,Manning,4256,331.p.4256,O,QB,https://sports.yahoo.com/nfl/players/4256,2014
3,4,K,nfl.p.4269,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['K'],small,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,False,False,False,0,Phil,Phil,Phil Dawson,Dawson,4269,331.p.4269,K,K,https://sports.yahoo.com/nfl/players/4269,2014
4,10,QB,nfl.p.4416,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,False,False,False,0,Matt,Matt,Matt Hasselbeck,Hasselbeck,4416,331.p.4416,O,QB,https://sports.yahoo.com/nfl/players/4416,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14037,12,WR,nfl.p.42519,LAC,Los Angeles Chargers,nfl.t.24,https://sports.yahoo.com/nfl/teams/la-chargers/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/VhPqpl4FF1X5...,https://s.yimg.com/iu/api/res/1.2/VhPqpl4FF1X5...,False,False,False,0,Dalevon,Dalevon,Dalevon Campbell,Campbell,42519,461.p.42519,O,WR,https://sports.yahoo.com/nfl/players/42519,2025
14044,11,QB,nfl.p.42537,NO,New Orleans Saints,nfl.t.18,https://sports.yahoo.com/nfl/teams/new-orleans/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/SclDLq4JOthk...,https://s.yimg.com/iu/api/res/1.2/SclDLq4JOthk...,False,False,False,0,Hunter,Hunter,Hunter Dekkers,Dekkers,42537,461.p.42537,O,QB,https://sports.yahoo.com/nfl/players/42537,2025
14047,5,WR,nfl.p.42545,GB,Green Bay Packers,nfl.t.9,https://sports.yahoo.com/nfl/teams/green-bay/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/duAZD8ehn2.t...,https://s.yimg.com/iu/api/res/1.2/duAZD8ehn2.t...,False,False,False,0,Sam,Sam,Sam Brown Jr.,Brown Jr.,42545,461.p.42545,O,WR,https://sports.yahoo.com/nfl/players/42545,2025
14052,9,RB,nfl.p.42562,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Toa,Toa,Toa Taua,Taua,42562,461.p.42562,O,RB,https://sports.yahoo.com/nfl/players/42562,2025


I will add the id in stats ID columns 

In [90]:
wefwefwefwefwefwefwefwe

NameError: name 'wefwefwefwefwefwefwefwe' is not defined

In [ ]:
yahoo_players[yahoo_players['name.full'].str.startswith('Andrew')].head(20)

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season
299,9,TE,nfl.p.24130,GB,Green Bay Packers,nfl.t.9,https://sports.yahoo.com/nfl/teams/green-bay/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/n1WorK4gEZDK...,https://s.yimg.com/iu/api/res/1.2/n1WorK4gEZDK...,False,False,False,0,Andrew,Andrew,Andrew Quarless,Quarless,24130,331.p.24130,O,TE,https://sports.yahoo.com/nfl/players/24130,2014
352,4,WR,nfl.p.24774,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/OMqXTfJd40ld...,https://s.yimg.com/iu/api/res/1.2/OMqXTfJd40ld...,False,False,False,0,Andrew,Andrew,Andrew Hawkins,Hawkins,24774,331.p.24774,O,WR,https://sports.yahoo.com/nfl/players/24774,2014
483,10,QB,nfl.p.25711,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/py29KAjMiTYt...,https://s.yimg.com/iu/api/res/1.2/py29KAjMiTYt...,False,False,False,0,Andrew,Andrew,Andrew Luck,Luck,25711,331.p.25711,O,QB,https://sports.yahoo.com/nfl/players/25711,2014
583,9,TE,nfl.p.26076,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/tG2QjQ7FE6UI...,https://s.yimg.com/iu/api/res/1.2/tG2QjQ7FE6UI...,False,False,False,0,Andrew,Andrew,Andrew Szczerba,Szczerba,26076,331.p.26076,O,TE,https://sports.yahoo.com/nfl/players/26076,2014
646,10,TE,nfl.p.26583,Min,Minnesota Vikings,nfl.t.16,https://sports.yahoo.com/nfl/teams/minnesota/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/f0hoO_Wz9sSq...,https://s.yimg.com/iu/api/res/1.2/f0hoO_Wz9sSq...,False,False,False,0,Andrew,Andrew,Andrew DePaola,DePaola,26583,331.p.26583,O,TE,https://sports.yahoo.com/nfl/players/26583,2014
1006,9,WR,nfl.p.28008,Det,Detroit Lions,nfl.t.8,https://sports.yahoo.com/nfl/teams/detroit/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/2TQwM3Oot.p2...,https://s.yimg.com/iu/api/res/1.2/2TQwM3Oot.p2...,False,False,False,0,Andrew,Andrew,Andrew Peacock,Peacock,28008,331.p.28008,O,WR,https://sports.yahoo.com/nfl/players/28008,2014
1125,11,K,nfl.p.28313,NYJ,New York Jets,nfl.t.20,https://sports.yahoo.com/nfl/teams/ny-jets/,['K'],small,https://s.yimg.com/iu/api/res/1.2/ilCNU_twT9fy...,https://s.yimg.com/iu/api/res/1.2/ilCNU_twT9fy...,False,False,False,0,Andrew,Andrew,Andrew Furney,Furney,28313,331.p.28313,K,K,https://sports.yahoo.com/nfl/players/28313,2014
1395,7,TE,nfl.p.24130,GB,Green Bay Packers,nfl.t.9,https://sports.yahoo.com/nfl/teams/green-bay/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/n1WorK4gEZDK...,https://s.yimg.com/iu/api/res/1.2/n1WorK4gEZDK...,False,False,False,0,Andrew,Andrew,Andrew Quarless,Quarless,24130,348.p.24130,O,TE,https://sports.yahoo.com/nfl/players/24130,2015
1429,11,WR,nfl.p.24774,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/OMqXTfJd40ld...,https://s.yimg.com/iu/api/res/1.2/OMqXTfJd40ld...,False,False,False,0,Andrew,Andrew,Andrew Hawkins,Hawkins,24774,348.p.24774,O,WR,https://sports.yahoo.com/nfl/players/24774,2015
1529,10,QB,nfl.p.25711,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/py29KAjMiTYt...,https://s.yimg.com/iu/api/res/1.2/py29KAjMiTYt...,False,False,False,0,Andrew,Andrew,Andrew Luck,Luck,25711,348.p.25711,O,QB,https://sports.yahoo.com/nfl/players/25711,2015


In [ ]:
master_ids.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
0,17030,3c76cab3-3df2-43dd-acaa-57e055bd32d0,24755.0,00-0040676,133244.0,12522.0,58203.0,4688380.0,41786,NaN,3168422.0,WardCa00,NaN,16997.0,NaN,1730.0,41786.0,0.0,25323.0,NaN,Cam Ward,cam ward,QB,TEN,2002-05-25,23.4,2025.0,1.0,1.0,1.0,NaN,74.0,219.0,Miami (FL),2025
1,17031,270e09bc-8bf8-44b0-87ed-1fd014de4ab7,25968.0,00-0040668,131396.0,12524.0,58349.0,4432762.0,41930,NaN,26710004.0,SandSh00,NaN,18479.0,NaN,1731.0,41930.0,0.0,26079.0,NaN,Shedeur Sanders,shedeur sanders,QB,CLE,2002-02-07,23.7,2025.0,5.0,6.0,144.0,NaN,74.0,212.0,Colorado,2025
2,17032,53848cba-bcad-4cd0-bb35-0a1f6ed111d1,23160.0,00-0040691,146409.0,12508.0,58227.0,4689114.0,41810,NaN,26694019.0,DartJa00,NaN,18574.0,NaN,1732.0,41810.0,0.0,26082.0,NaN,Jaxson Dart,jaxson dart,QB,NYG,2003-05-13,22.4,2025.0,1.0,25.0,25.0,NaN,74.0,223.0,Ole Miss,2025
3,17033,b0eb5b67-4ef2-462d-bfe7-e73bcce1f6b0,22884.0,00-0040673,146590.0,12510.0,58297.0,4432734.0,41882,NaN,26694943.0,MilrJa00,NaN,18477.0,NaN,1733.0,41882.0,0.0,25882.0,NaN,Jalen Milroe,jalen milroe,QB,SEA,2002-12-13,22.8,2025.0,3.0,28.0,92.0,NaN,74.0,216.0,Alabama,2025
4,17034,d89f0384-21a6-44e4-8aa6-324cb2103bf6,23680.0,00-0040234,127253.0,12500.0,58436.0,4889929.0,42037,NaN,26746721.0,EwerQu00,NaN,18505.0,NaN,1734.0,42037.0,0.0,26048.0,NaN,Quinn Ewers,quinn ewers,QB,MIA,2003-05-13,22.4,2025.0,7.0,15.0,231.0,NaN,74.0,209.0,Texas,2025


In [ ]:
dump_players = master_ids[~master_ids['yahoo_id'].isin(yahoo_players['player_id'])]

In [ ]:
master_ids = master_ids.drop(dump_players, axis = 1)

In [ ]:
master_ids.head()

""
0
1
2
3
4


In [ ]:
master_ids[~master_ids['yahoo_id'].isin(yahoo_players['player_id'])]

KeyError: 'yahoo_id'

In [ ]:
yahoo_players[yahoo_players['player_id'].isin(master_ids['yahoo_id'])]

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season
0,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2014
1,9,TE,nfl.p.3950,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,False,False,False,0,Tony,Tony,Tony Gonzalez,Gonzalez,3950,331.p.3950,O,TE,https://sports.yahoo.com/nfl/players/3950,2014
2,4,QB,nfl.p.4256,Den,Denver Broncos,nfl.t.7,https://sports.yahoo.com/nfl/teams/denver/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,False,False,False,0,Peyton,Peyton,Peyton Manning,Manning,4256,331.p.4256,O,QB,https://sports.yahoo.com/nfl/players/4256,2014
3,4,K,nfl.p.4269,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['K'],small,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,False,False,False,0,Phil,Phil,Phil Dawson,Dawson,4269,331.p.4269,K,K,https://sports.yahoo.com/nfl/players/4269,2014
4,10,QB,nfl.p.4416,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,False,False,False,0,Matt,Matt,Matt Hasselbeck,Hasselbeck,4416,331.p.4416,O,QB,https://sports.yahoo.com/nfl/players/4416,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14044,11,QB,nfl.p.42537,NO,New Orleans Saints,nfl.t.18,https://sports.yahoo.com/nfl/teams/new-orleans/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/SclDLq4JOthk...,https://s.yimg.com/iu/api/res/1.2/SclDLq4JOthk...,False,False,False,0,Hunter,Hunter,Hunter Dekkers,Dekkers,42537,461.p.42537,O,QB,https://sports.yahoo.com/nfl/players/42537,2025
14047,5,WR,nfl.p.42545,GB,Green Bay Packers,nfl.t.9,https://sports.yahoo.com/nfl/teams/green-bay/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/duAZD8ehn2.t...,https://s.yimg.com/iu/api/res/1.2/duAZD8ehn2.t...,False,False,False,0,Sam,Sam,Sam Brown Jr.,Brown Jr.,42545,461.p.42545,O,WR,https://sports.yahoo.com/nfl/players/42545,2025
14052,9,RB,nfl.p.42562,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Toa,Toa,Toa Taua,Taua,42562,461.p.42562,O,RB,https://sports.yahoo.com/nfl/players/42562,2025
14061,9,WR,nfl.p.42598,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Isaiah,Isaiah,Isaiah Bond,Bond,42598,461.p.42598,O,WR,https://sports.yahoo.com/nfl/players/42598,2025


In [144]:
def position_checker():
    master_copy = master_raw.copy()
    master_copy.loc[(master_copy['yahoo_id'].isna()) & (master_copy['stats_id'].notna()), 'yahoo_id'] = master_copy.loc[(master_copy['yahoo_id'].isna()) & (master_copy['stats_id'].notna()), 'stats_id']
    full_copy = yahoo_players.merge(master_copy, how='left', left_on='player_id', right_on='yahoo_id')
    mismatches = full_copy[full_copy['primary_position'] != full_copy['position']]
    return mismatches

In [147]:
result = position_checker()

In [151]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2275 entries, 69 to 14104
Data columns (total 60 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   bye_weeks.week            2275 non-null   int64  
 1   display_position          2275 non-null   object 
 2   editorial_player_key      2275 non-null   object 
 3   editorial_team_abbr       2275 non-null   object 
 4   editorial_team_full_name  2275 non-null   object 
 5   editorial_team_key        2275 non-null   object 
 6   editorial_team_url        2275 non-null   object 
 7   eligible_positions        2275 non-null   object 
 8   headshot.size             2275 non-null   object 
 9   headshot.url              2275 non-null   object 
 10  image_url                 2275 non-null   object 
 11  is_keeper.status          2275 non-null   bool   
 12  is_keeper.cost            2275 non-null   bool   
 13  is_keeper.kept            2275 non-null   bool   
 14  is_undroppa

In [153]:
result.head(50)

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
69,9,K,nfl.p.7434,Ten,Tennessee Titans,nfl.t.10,https://sports.yahoo.com/nfl/teams/tennessee/,['K'],small,https://s.yimg.com/iu/api/res/1.2/etIu4LtcHSr7...,https://s.yimg.com/iu/api/res/1.2/etIu4LtcHSr7...,False,False,False,0,Rob,Rob,Rob Bironas,Bironas,7434,331.p.7434,K,K,https://sports.yahoo.com/nfl/players/7434,2014,8074.0,8e8b22b2-9c58-4a24-9d5a-daded760e1f3,NaN,00-0020962,1218.0,NaN,2504934.0,9235.0,7434.0,NaN,303026.0,bironrob01,NaN,3527.0,3419.0,NaN,7434.0,250445.0,NaN,NaN,Rob Bironas,rob bironas,XX,FA,1978-01-29,43.3,2001.0,NaN,NaN,NaN,RobBironas,72.0,208.0,Georgia Southern,2025.0
182,8,RB,nfl.p.9004,NYG,New York Giants,nfl.t.19,https://sports.yahoo.com/nfl/teams/ny-giants/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/xlu80ZICCzpp...,https://s.yimg.com/iu/api/res/1.2/xlu80ZICCzpp...,False,False,False,0,Peyton,Peyton,Peyton Hillis,Hillis,9004,331.p.9004,O,RB,https://sports.yahoo.com/nfl/players/9004,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,4,RB,nfl.p.9039,Cin,Cincinnati Bengals,nfl.t.4,https://sports.yahoo.com/nfl/teams/cincinnati/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/e76NOWDVv5.v...,https://s.yimg.com/iu/api/res/1.2/e76NOWDVv5.v...,False,False,False,0,BenJarvus,BenJarvus,BenJarvus Green-Ellis,Green-Ellis,9039,331.p.9039,O,RB,https://sports.yahoo.com/nfl/players/9039,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
348,12,WR,nfl.p.24746,Car,Carolina Panthers,nfl.t.29,https://sports.yahoo.com/nfl/teams/carolina/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/M8gJ9e0k7poq...,https://s.yimg.com/iu/api/res/1.2/M8gJ9e0k7poq...,False,False,False,0,Teddy,Teddy,Teddy Williams,Williams,24746,331.p.24746,O,WR,https://sports.yahoo.com/nfl/players/24746,2014,10233.0,09d857ac-02ec-4a93-86af-1f8384386dc8,NaN,00-0027853,6073.0,781.0,2508077.0,13922.0,24746.0,NaN,1758984.0,WillTe00,NaN,8581.0,6406.0,NaN,24746.0,556603.0,12643.0,NaN,Teddy Williams,teddy williams,CB,CAR,1988-07-03,32.8,2010.0,NaN,NaN,NaN,_FastTeddy_,73.0,210.0,Texas-San Antonio,2025.0
415,11,RB,nfl.p.24998,Jax,Jacksonville Jaguars,nfl.t.30,https://sports.yahoo.com/nfl/teams/jacksonville/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/4MMC561y.aX9...,https://s.yimg.com/iu/api/res/1.2/4MMC561y.aX9...,False,False,False,0,Bruce,Bruce,Bruce Miller,Miller,24998,331.p.24998,O,RB,https://sports.yahoo.com/nfl/players/24998,2014,10437.0,9be8224a-4a19-4f6a-a2be-ecbd3a24868c,10031.0,00-0028149,6363.0,982.0,2495314.0,14083.0,24998.0,NaN,1114232.0,MillBr02,bruce-miller-1,7505.0,6713.0,NaN,24998.0,324654.0,13446.0,NaN,Bruce Miller,bruce miller,TE,FA,1987-08-06,33.8,2011.0,7.0,8.0,211.0,BruceMill49er,74.0,248.0,Central Florida,2025.0
423,4,TE,nfl.p.25050,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/SKdKxY1l7uWA...,https://s.yimg.com/iu/api/res/1.2/SKdKxY1l7uWA...,False,False,False,0,Martell,Martell,Martell Webb,Webb,25050,331.p.25050,O,TE,https://sports.yahoo.com/nfl/players/25050,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [149]:
result['position'].value_counts()

position
TE    110
RB     44
CB     24
WR     22
LB     11
PN      9
S       4
XX      3
DL      3
QB      3
DT      2
DE      1
Name: count, dtype: int64